# **Uniform Random 3-SAT Problem**

In [ ]:
import random

def generate_3_sat(m, n):
    clauses = []
    for _ in range(m):
        clause = set()
        while len(clause) < 3:
            var = random.randint(1, n)
            if random.choice([True, False]):
                var = -var  # Negate the variable
            clause.add(var)
        clauses.append(list(clause))
    return clauses

def evaluate_solution(clauses, solution):
    # Heuristic h1: Number of satisfied clauses.
    satisfied_clauses = 0
    for clause in clauses:
        for literal in clause:
            if literal > 0 and solution[abs(literal) - 1] == 1:
                satisfied_clauses += 1
                break
            elif literal < 0 and solution[abs(literal) - 1] == 0:
                satisfied_clauses += 1
                break
    return satisfied_clauses

def unsatisfied_clauses(clauses, solution):
    # Heuristic h2: Number of unsatisfied clauses.
    unsatisfied = 0
    for clause in clauses:
        clause_satisfied = False
        for literal in clause:
            if literal > 0 and solution[abs(literal) - 1] == 1:
                clause_satisfied = True
                break
            elif literal < 0 and solution[abs(literal) - 1] == 0:
                clause_satisfied = True
                break
        if not clause_satisfied:
            unsatisfied += 1
    return unsatisfied

# **Hill Climbing**

In [ ]:
def hill_climbing(clauses, n, max_iterations=1000):
    current_solution = [random.randint(0, 1) for _ in range(n)]
    current_value = evaluate_solution(clauses, current_solution)

    for _ in range(max_iterations):
        neighbor = current_solution[:]
        # Flip a random bit
        var_to_flip = random.randint(0, n - 1)
        neighbor[var_to_flip] = 1 - neighbor[var_to_flip]

        neighbor_value = evaluate_solution(clauses, neighbor)
        if neighbor_value > current_value:
            current_solution = neighbor
            current_value = neighbor_value

        if current_value == len(clauses):
            break

    return current_solution, current_value

# **Beam Search**

In [ ]:
def beam_search(clauses, n, beam_width, max_iterations=1000):
    beams = [[random.randint(0, 1) for _ in range(n)] for _ in range(beam_width)]
    best_solution = None
    best_value = 0

    for _ in range(max_iterations):
        next_beams = []
        for beam in beams:
            for i in range(n):
                neighbor = beam[:]
                neighbor[i] = 1 - neighbor[i]
                next_beams.append(neighbor)

        next_beams = sorted(next_beams, key=lambda b: evaluate_solution(clauses, b), reverse=True)
        beams = next_beams[:beam_width]

        current_best_value = evaluate_solution(clauses, beams[0])
        if current_best_value > best_value:
            best_solution = beams[0]
            best_value = current_best_value

        if best_value == len(clauses):
            break

    return best_solution, best_value

# **Variable-Neighborhood-Descent with 3 Neighborhood**

In [ ]:
def variable_neighborhood_descent(clauses, n, max_iterations=1000):
    def neighborhood_1(solution):
        # Flip one variable
        neighbors = []
        for i in range(n):
            neighbor = solution[:]
            neighbor[i] = 1 - neighbor[i]
            neighbors.append(neighbor)
        return neighbors

    def neighborhood_2(solution):
        # Flip two variables
        neighbors = []
        for i in range(n):
            for j in range(i + 1, n):
                neighbor = solution[:]
                neighbor[i] = 1 - neighbor[i]
                neighbor[j] = 1 - neighbor[j]
                neighbors.append(neighbor)
        return neighbors

    def neighborhood_3(solution):
        # Flip three variables
        neighbors = []
        for i in range(n):
            for j in range(i + 1, n):
                for k in range(j + 1, n):
                    neighbor = solution[:]
                    neighbor[i] = 1 - neighbor[i]
                    neighbor[j] = 1 - neighbor[j]
                    neighbor[k] = 1 - neighbor[k]
                    neighbors.append(neighbor)
        return neighbors

    neighborhoods = [neighborhood_1, neighborhood_2, neighborhood_3]
    current_solution = [random.randint(0, 1) for _ in range(n)]
    current_value = evaluate_solution(clauses, current_solution)

    for _ in range(max_iterations):
        improved = False
        for neighborhood in neighborhoods:
            neighbors = neighborhood(current_solution)
            best_neighbor = max(neighbors, key=lambda s: evaluate_solution(clauses, s))
            best_neighbor_value = evaluate_solution(clauses, best_neighbor)
            if best_neighbor_value > current_value:
                current_solution = best_neighbor
                current_value = best_neighbor_value
                improved = True
                break

        if not improved:
            break

    return current_solution, current_value

In [ ]:
def experiment(m_values, n_values, repetitions=5):
    algorithms = [
        ('Hill Climbing', hill_climbing),
        ('Beam Search (width=3)', lambda c, n: beam_search(c, n, beam_width=3)),
        ('Beam Search (width=4)', lambda c, n: beam_search(c, n, beam_width=4)),
        ('VND', variable_neighborhood_descent),
    ]

    for m in m_values:
        for n in n_values:
            print(f"\nTesting with m={m}, n={n}:")
            for algo_name, algo in algorithms:
                success_count = 0
                for _ in range(repetitions):
                    clauses = generate_3_sat(m, n)
                    solution, value = algo(clauses, n)
                    if value == len(clauses):
                        success_count += 1
                penetrance = success_count / repetitions
                print(f"{algo_name} penetrance : {penetrance * 100:.2f}%")

m_values = [20, 40, 60]
n_values = [10, 20, 30]
experiment(m_values, n_values)


Testing with m=20, n=10:
Hill Climbing penetrance : 100.00%
Beam Search (width=3) penetrance : 100.00%
Beam Search (width=4) penetrance : 100.00%
VND penetrance : 100.00%

Testing with m=20, n=20:
Hill Climbing penetrance : 100.00%
Beam Search (width=3) penetrance : 100.00%
Beam Search (width=4) penetrance : 100.00%
VND penetrance : 100.00%

Testing with m=20, n=30:
Hill Climbing penetrance : 100.00%
Beam Search (width=3) penetrance : 100.00%
Beam Search (width=4) penetrance : 100.00%
VND penetrance : 100.00%

Testing with m=40, n=10:
Hill Climbing penetrance : 20.00%
Beam Search (width=3) penetrance : 80.00%
Beam Search (width=4) penetrance : 100.00%
VND penetrance : 100.00%

Testing with m=40, n=20:
Hill Climbing penetrance : 80.00%
Beam Search (width=3) penetrance : 100.00%
Beam Search (width=4) penetrance : 100.00%
VND penetrance : 100.00%

Testing with m=40, n=30:
Hill Climbing penetrance : 100.00%
Beam Search (width=3) penetrance : 100.00%
Beam Search (width=4) penetrance : 100.